# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
pf.__version__

'0.0.51'

In [16]:
import os
DayFolder=os.getcwd()
meta_path=DayFolder +  '\MetaData'
spectra_path=DayFolder + '\Spectra'

## Load in specra, and combine

In [17]:
import os.path
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [18]:
df2

,Unnamed: 0,filename,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,Cap19-05X1,102.805533,1286.621108,224.870112,1286.621158,337.974447,0.632936,1.712240,3.211298e-01,...,0.592090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Cap19-05X2,102.759511,1286.620738,207.881715,1286.620788,295.229813,0.642750,2.221874,1.132346e-01,...,0.615226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Cap19-06X1,102.977382,1286.142245,917.359532,1286.142295,1677.688548,0.722776,2.472211,4.923364e-01,...,0.858906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Cap19-06X2,102.941170,1286.176076,1178.920275,1286.176126,2047.286829,0.703560,3.024881,4.267956e-01,...,0.747058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Cap19-07X1,102.735556,1286.700138,45.642781,1286.700138,69.634145,0.715356,1.685106,8.742618e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,0,POC5,104.649314,1282.314885,28671.259230,1282.314835,84679.164965,1.082499,32.418334,7.049583e-01,...,2.498854,1369.584030,1792.680106,0.501223,1385.642293,12844.760362,15.466267,1285.672350,7531.498445,11.631539
43,0,POC6,104.592831,1282.422510,27808.764318,1282.422460,82328.189801,1.086695,31.608597,6.998413e-01,...,2.400701,1369.585673,1742.326264,0.498119,1385.695019,12453.367406,16.101664,1286.369226,6887.526994,11.680501
44,0,POC7,104.528352,1282.556610,26093.143533,1282.556560,77623.594987,1.096656,34.607899,6.880825e-01,...,2.358888,1369.602956,1619.006792,0.494994,1384.670181,10676.199496,15.798928,1287.499095,6594.863361,12.514597
45,0,POC8,104.441870,1282.731033,14917.983251,1282.730983,44962.480164,1.107162,17.609707,6.929906e-01,...,2.272979,1369.626874,918.719215,0.488770,1383.428146,5755.988201,16.014974,1288.021186,3067.450120,12.532091


## First, get the metadata for this day

In [19]:
help(pf.get_files)

Help on function get_files in module DiadFit.importing_data_files:

get_files(path, ID_str=None, file_ext='txt', exclude_str=None, exclude_type=None, sort=True)
    This function takes a user path, and extracts all files which contain the ID_str
    
    Parameters
    -----------
    
    path: str
        Folder user wishes to read data from
    sort: bool
        If true, sorts files alphabetically
    ID_str: list
        Finds all files containing this string (e.g. ['Ne', 'NE']
    exclude_str: str
        Excludes files with this string in the name
    file_ext: str
        Gets all files of this format only (e.g. txt)
    
    
    Returns
    -----------
    Returns file names as a list.



In [20]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'FG','Cap','window'],
            exclude_type='.png',   sort=True)
diad_meta

['POC1.txt',
 'POC10.txt',
 'POC11.txt',
 'POC12.txt',
 'POC13.txt',
 'POC14.txt',
 'POC15.txt',
 'POC16.txt',
 'POC17.txt',
 'POC18.txt',
 'POC19.txt',
 'POC2.txt',
 'POC20.txt',
 'POC21.txt',
 'POC22.txt',
 'POC23.txt',
 'POC24.txt',
 'POC25.txt',
 'POC26.txt',
 'POC27.txt',
 'POC28.txt',
 'POC29.txt',
 'POC3.txt',
 'POC30.txt',
 'POC31.txt',
 'POC32.txt',
 'POC33.txt',
 'POC34.txt',
 'POC35.txt',
 'POC36.txt',
 'POC37.txt',
 'POC38.txt',
 'POC39.txt',
 'POC4.txt',
 'POC40.txt',
 'POC41.txt',
 'POC42.txt',
 'POC43.txt',
 'POC44.txt',
 'POC45.txt',
 'POC46.txt',
 'POC47.txt',
 'POC48.txt',
 'POC49.txt',
 'POC5.txt',
 'POC50.txt',
 'POC51.txt',
 'POC52.txt',
 'POC53.txt',
 'POC54.txt',
 'POC55.txt',
 'POC56.txt',
 'POC57-0.04.txt',
 'POC58.txt',
 'POC6.txt',
 'POC7.txt',
 'POC8.txt',
 'POC9.txt']

## Now get all the important information out of the metadata files

In [21]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=False)
meta['filename'].iloc[0]

100%|██████████████████████████████████████████████████████████████████████████████████| 58/58 [00:02<00:00, 25.40it/s]

Done
Oof, try not to work after midnight!


'POC49.txt'

## Then get a simple file name you can stitch with the spectra

In [22]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

'POC49'

## Now get filename from spectra into same form as metadata

In [23]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

file_s

duplicates
73     FG04_31_MI3 (1)
74         FG04_31_MI3
75     FG04_31_MI4 (1)
77               POC10
78               POC11
83               POC16
84               POC17
85               POC18
86               POC19
88               POC20
89               POC21
90               POC22
91               POC23
92               POC24
93               POC26
94               POC27
95               POC28
96               POC29
98               POC30
99               POC31
100              POC32
101              POC33
102              POC34
103              POC35
104              POC36
105              POC37
106              POC38
107              POC39
109              POC40
110              POC41
111              POC42
112              POC43
113              POC44
114              POC45
dtype: object
OOPS. at least one of your file name is duplicated go back to your spectra, you named a file twice, this will confuse the stitching 


array(['Cap19-05X1', 'Cap19-05X2', 'Cap19-06X1', 'Cap19-06X2',
       'Cap19-07X1', 'Cap19-07X2', 'Cap19-08X1', 'Cap19-08X2',
       'Cap19-12X1', 'Cap19-12X2', 'Cap19-17X1', 'Cap19-17X2',
       'Cap19-18X1', 'Cap19-18X2', 'FG04_31_MI', 'FG04_31_MI4', 'POC25',
       'POC46', 'POC47', 'POC48', 'POC49', 'POC50 (Sub BG)', 'POC50',
       'POC51 (Sub BG)', 'POC51', 'POC52 (Sub BG)', 'POC52',
       'POC53 (Sub BG)', 'POC53', 'POC54 (Sub BG)', 'POC54',
       'POC55 (Sub BG)', 'POC55', 'POC56 (Sub BG)', 'POC56',
       'POC57-0.04 (Sub BG)', 'POC57-0.04', 'POC58 (Sub BG)', 'POC58',
       'FG04_31_MI3 (1)', 'FG04_31_MI3', 'FG04_31_MI4 (1)', 'POC10',
       'POC11', 'POC16', 'POC17', 'POC18', 'POC19', 'POC20', 'POC21',
       'POC22', 'POC23', 'POC24', 'POC26', 'POC27', 'POC28', 'POC29',
       'POC30', 'POC31', 'POC32', 'POC33', 'POC34', 'POC35', 'POC36',
       'POC37', 'POC38', 'POC39', 'POC40', 'POC41', 'POC42', 'POC43',
       'POC44', 'POC45', 'FG04_31_MI3 (1)', 'FG04_31_MI3',
      

In [24]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [25]:
from pickle import load
import pickle
import os
pickle_path=os.getcwd()
file_pkl=pickle_path + '/' + 'Ne_Line_Regression_Example_cali.pck'
with open(file_pkl, 'rb') as file_handle:
    my_loaded_model = pickle.load(file_handle)

In [13]:
df_combo['Ne_Correction']=my_loaded_model(df_combo['sec since midnight'])
df_combo.insert(1,'Corrected_Splitting', df_combo['Ne_Correction']*df_combo['Splitting'])

In [14]:
df_combo.to_excel('Calibration_fitting.xlsx')